<a href="https://colab.research.google.com/github/AsadiAhmad/Word-Predictor/blob/main/Code/Word_Predictor_N_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import Libraries

In [ ]:
import requests

# Step 2: Get Corpus

In [ ]:
train_set_url = "https://raw.githubusercontent.com/AsadiAhmad/Word-Predictor/refs/heads/main/Corpus/tarzan.txt"
